In [ ]:
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import pandas as pd
import time
import yfinance as yf

In [ ]:
class Portfolio:
    def __init__(self, initial_cash, broker_fee=0.002):
        self.initial_cash = initial_cash # probably won't have to use this variable
        self.remaining_cash = initial_cash
        self.broker_fee = broker_fee
        self.shares_owned = 0
        self.metrics_df = pd.DataFrame(columns=['Shares Owned', 'Cash Held', 'Profits'])
    
    # lingering question: how to have this if an order has different prices in it?
    def buy(self, buy_shares, buy_price, buy_time):
        # make sure we have the money to buy the shares we want
        if self.remaining_cash - (buy_shares * buy_price) < 0:
            print('Not enough money to fill this order')
        else:
            self.remaining_cash -= ((buy_shares * buy_price) + (buy_shares * buy_price * self.broker_fee))
            self.shares_owned += buy_shares
            print(f'We are buying {buy_shares} shares at ${buy_price} per share!')

            # add this new position to a dataframe of metrics
            self.metrics_df.loc[buy_time] = {'Shares Owned': self.shares_owned, 'Cash Held': self.remaining_cash, 'Profits': self.remaining_cash - self.initial_cash}
    
    def sell(self, sell_shares, sell_price, sell_time):
        # make sure we have enough shares to sell the amount we want to sell
        if self.shares_owned < sell_shares:
            print('Not enough shares held to fill this order')
        else:
            self.remaining_cash += ((sell_shares * sell_price) - (sell_shares * sell_price * self.broker_fee))
            self.shares_owned -= sell_shares
            print(f'We are selling {sell_shares} shares at ${sell_price} per share!')
            
            # add this new position to a dataframe of metrics
            self.metrics_df.loc[sell_time] = {'Shares Owned': self.shares_owned, 'Cash Held': self.remaining_cash, 'Profits': self.remaining_cash - self.initial_cash}
    
    def show_metrics(self):
        print(self.metrics_df)
    
    def metrics_graph(self):

        # Plot y1 and y2 on the primary y-axis, and y3 on the secondary y-axis
        fig, ax1 = plt.subplots(figsize=(10, 6))  # Adjust the figure size if needed

        # Setting the x-axis
        ax1.set_xlabel('Date & Time')
        ax1.tick_params(axis='x', rotation=45)
        ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S')) 

        # Plot y1 and y2 on the primary y-axis
        ax1.set_ylabel('Cash in USD')
        ax1.plot(self.metrics_df.index, self.metrics_df['Cash Held'], label='Cash Held', color='tab:blue', linestyle='-')
        ax1.plot(self.metrics_df.index, self.metrics_df['Profits'], label='Profits', color='tab:green', linestyle='--')

        # Format the numbers on the y axis to be in dollars
        ax1.yaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}')) 
        
        # Create a secondary y-axis for y3
        ax2 = ax1.twinx()
        ax2.set_ylabel('Number of Shares')
        ax2.plot(self.metrics_df.index, self.metrics_df['Shares Owned'], label='Shares Owned', color='tab:red', linestyle='-.')

        # Make the line at y=0 bolder
        ax1.axhline(y=0, color='black', linewidth=2)
        
        # Add title to the plot
        plt.title('Trading Metrics Over Time')
        
        # Add legend
        fig.tight_layout()
        fig.legend(loc='upper right')

        # Show plot
        plt.grid(True)
        plt.show()

In [ ]:
def signal_gen(prev, past):
    signal_results = {'Action': 'Neutral', 'Shares': 0}
    if prev['Open'] - prev['Close'] < 0:
        signal_results['Action'] = 'Buy'
    else:
        signal_results['Action'] = 'Sell'
    
    signal_results['Shares'] = 1
    
    return signal_results

In [ ]:
msft = yf.Ticker('MSFT')

# get historical market data
today = datetime.now().strftime('%Y-%m-%d')
hist = msft.history(period='max', end=today, interval='1m') # max is 7 days when using 1 minute data

In [ ]:
# instantiate portfolio
p1 = Portfolio(1_000)

# this would actually be running the engine - getting data and seeing if buy or sell
for i in range(1, len(hist.iloc[:20])):
    prev_interval = hist.iloc[i-1]
    full_past = hist.iloc[:i] # slice to i because it is not inclusive
    
    signal = signal_gen(prev_interval, full_past)
    
    if signal['Action'] == 'Buy':
        p1.buy(signal['Shares'], prev_interval['High'], prev_interval.name)
    else:
        p1.sell(signal['Shares'], prev_interval['Low'], prev_interval.name)

In [ ]:
p1.show_metrics()
p1.metrics_graph()